In [109]:
import pandas as pd
import numpy as np
import requests
import json
import xlsxwriter
import math
from secrets import IEX_CLOUD_API_TOKEN

In [110]:
stocks=pd.read_csv('sp_500_stocks.csv')

In [111]:
col=['Ticker','latestPrice','marketCap','NumberOfSHaresToBuy']

stocks=pd.DataFrame(stocks,columns=['Ticker','latestPrice','marketCap','NumberOfSHaresToBuy'])

# stocks.columns=['latestPrice','MarketCap','NumberOfSHaresToBuy']
stocks.tail()

,Ticker,latestPrice,marketCap,NumberOfSHaresToBuy
500,YUM,NaN,NaN,NaN
501,ZBH,NaN,NaN,NaN
502,ZBRA,NaN,NaN,NaN
503,ZION,NaN,NaN,NaN
504,ZTS,NaN,NaN,NaN


In [112]:
symbol='AAPL'
url=F"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}"

In [113]:
print(url)
r=requests.get(url).json()

https://sandbox.iexapis.com/stable/stock/AAPL/quote/?token=Tpk_059b97af715d417d9f49f50b51b1c448


USING BATCH PROCESSING:


In [114]:
def chunks(lst,n):
    for i in range(0,len(lst),n):
        yield lst[i:i+n]

In [115]:
symbol_groups=list(chunks(stocks['Ticker'],100))

In [116]:
symbol_groups
symbol_strings=[]
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

In [117]:
stocks_bp=pd.DataFrame(columns=col)

In [118]:
stocks_bp.head()

,Ticker,latestPrice,marketCap,NumberOfSHaresToBuy


In [119]:
stocks_bp.Ticker=pd.Series(pd.concat(symbol_groups))


In [120]:
batch=[]
for s in symbol_strings:
    url=f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={s}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    r=requests.get(url).json()
    batch.append(r)

In [121]:
dp={}
dm={}
for i in range(len(batch)):
    for key in batch[i].keys():
        dp[key]=batch[i][key]['quote']['latestPrice']
        dm[key]=batch[i][key]['quote']['marketCap']
len(batch)

6

In [122]:
s=pd.Series(dp.values())
m=pd.Series(dm.values())

In [123]:
stocks_bp.latestPrice=s
stocks_bp.marketCap=m

In [124]:
stocks_bp.index=stocks_bp.Ticker

In [125]:
stocks_bp.drop('Ticker',axis=1,inplace=True)

In [126]:
for t in stocks_bp.index:
    if t in dp.keys():
        stocks_bp.loc[t,'latestPrice']=dp[t]
        stocks_bp.loc[t,'marketCap']=dm[t]

In [127]:
stocks_bp.NumberOfSHaresToBuy=stocks_bp.NumberOfSHaresToBuy.fillna('N/A')


In [128]:
stocks_bp.to_csv('stocks_bp.csv')

In [129]:
stocks_bp

,latestPrice,marketCap,NumberOfSHaresToBuy
Ticker,,,
A,118.380,3.419712e+10,N/A
AAL,13.150,8.401792e+09,N/A
AAP,171.680,1.034117e+10,N/A
AAPL,136.856,2.151772e+12,N/A
ABBV,144.181,2.434140e+11,N/A
...,...,...,...
YUM,110.690,3.157973e+10,N/A
ZBH,106.110,2.255252e+10,N/A
ZBRA,301.360,1.549958e+10,N/A


In [130]:

stocks_bp.head()

,latestPrice,marketCap,NumberOfSHaresToBuy
Ticker,,,
A,118.380,3.419712e+10,N/A
AAL,13.150,8.401792e+09,N/A
AAP,171.680,1.034117e+10,N/A
AAPL,136.856,2.151772e+12,N/A
ABBV,144.181,2.434140e+11,N/A


In [131]:
stocks_bp.loc['AAPL']

latestPrice                    136.856
marketCap              2151771687545.0
NumberOfSHaresToBuy                N/A
Name: AAPL, dtype: object

In [132]:
portfolio=input('Enter the value of your portfolio: ')
try:
    portfolio=float(portfolio)
except ValueError:
    print("not int")
    portfolio=input("Enter the value of your portfolio:")
    portfolio=float(portfolio)
else:
    print(portfolio)


Enter the value of your portfolio: 10000000
10000000.0


In [133]:
position_size=portfolio/len(stocks_bp.index)

In [134]:
number_of_shares=position_size/stocks_bp.loc['AAPL','latestPrice']

In [135]:
number_of_shares

144.6920865582788

In [136]:
def calculate_number_of_shares(portfolio,stock):
    position_size=portfolio/len(stocks_bp.index)
    return position_size/stocks_bp.loc[stock,'latestPrice']

In [171]:
for stock in stocks_bp.index:
    stocks_bp.loc[stock,"NumberOfSHaresToBuy"]=math.floor(calculate_number_of_shares(portfolio,stock))
    

In [172]:
stocks_bp['SYMBOL']=stocks_bp.index
col_list = list(stocks)
col_list[0], col_list[1] = col_list[1], col_list[0]
stocks.head()

,Ticker,latestPrice,marketCap,NumberOfSHaresToBuy
0,A,NaN,NaN,NaN
1,AAL,NaN,NaN,NaN
2,AAP,NaN,NaN,NaN
3,AAPL,NaN,NaN,NaN
4,ABBV,NaN,NaN,NaN


In [163]:






writer=pd.ExcelWriter('RecommendedTrades.xlsx',engine='xlsxwriter')
stocks_bp.to_excel(writer,'RecommendedTrades',index=False)


In [164]:
backGround='#0a0a23'
font='#ffffff'

string_format=writer.book.add_format({
    'font_color': font,
    'bg_color':backGround,
    'border':1
})

dollar_format=writer.book.add_format({
    'num_format':'$0.00',
    'font_color': font,
    'bg_color':backGround,
    'border':1
})

integer_format=writer.book.add_format({
    'font_color': font,
    'num_format':'0',

    'bg_color':backGround,
    'border':1
})



In [165]:
# writer.sheets['RecommendedTrades'].set_column('A:A',18,string_format)
# writer.sheets['RecommendedTrades'].set_column('B:B',18,string_format)

# writer.sheets['RecommendedTrades'].set_column('C:C',18,string_format)
# writer.sheets['RecommendedTrades'].set_column('D:D',18,string_format)

writer.sheets['RecommendedTrades'].write('A1','latestPrice',dollar_format)
writer.sheets['RecommendedTrades'].write('B1','MarketCap',dollar_format)
writer.sheets['RecommendedTrades'].write('C1','Number of shares',integer_format)
writer.sheets['RecommendedTrades'].write('D1','Symbol',string_format)

0

In [166]:
column_formats={
    'A':['latestPrice',dollar_format],
    'B':['marketCap',dollar_format],
    'C':['NumberOfSharesToBuy',integer_format],
    
    'D':['SYMBOL',string_format],
}
for column in column_formats.keys():
    writer.sheets['RecommendedTrades'].set_column(f'{column}:{column}',18,column_formats[column][1])
writer.save()